In [1]:
import numpy as np
import matplotlib.pylab as plt
import pickle as pkl

import sys, os
sys.path.append(os.pardir) # sys에 parent directory를 설정해서 상위 폴더의 library도 import 가능
from dataset.mnist import load_mnist # 상위폴더의 dataset/mnist.py 내의 load_mnist 함수 import
from PIL import Image # 파이썬 자체의 이미지 처리 라이브러리

In [2]:
# Step_function

def step_function(x):
    return np.array(x > 0, dtype = np.int)

In [3]:
# # step graph
# x = np.arange(-5.0, 5.0, 0.1)
# y = step_function(x)

# plt.plot(x, y)
# plt.ylim(-0.1, 1.1)
# plt.show()

In [4]:
# Sigmoid_function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [5]:
# # Sigmoid_graph
# x = np.arange(-5.0, 5.0, 0.1)
# y = sigmoid(x)

# plt.plot(x, y)
# plt.show()

In [6]:
# Relu function
def relu(x):
    return np.maximum(0, x)

In [7]:
# # Relu graph

# x = np.arange(-5.0, 5.0, 0.1)
# y = relu(x)

# plt.plot(x, y)
# plt.ylim(-0.5, 5)
# plt.show()

In [8]:
# Identity function
def identity_function(x):
    return x

In [9]:
# Weight initialize

def init_network():
    network = {}
    network['W1'] = np.array([[0.1, 0.3, 0.5], [0.2, 0.4, 0.6]])
    network['b1'] = np.array([0.1, 0.2, 0.3])
    network['W2'] = np.array([[0.1, 0.4], [0.2, 0.5], [0.3, 0.6]])
    network['b2'] = np.array([0.1, 0.2])
    network['W3'] = np.array([[0.1, 0.3], [0.2, 0.4]])
    network['b3'] = np.array([0.1, 0.2])
    
    return network

In [10]:
# Forward network

def forward(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = identity_function(a3)
    
    return y

In [11]:
# Softmax function

def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c) # for overflow handling
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

In [12]:
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, flatten = True, one_hot_label = False)
    return x_test, t_test

In [13]:
def init_network(): # 초기 가중치는 이미 학습이 완료된 가중치를 가져옴.
    with open("ch03\sample_weight.pkl", 'rb') as f:
        network = pkl.load(f)
    return network

In [14]:
def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    
    return y

In [15]:
def img_show(img):
    plt.figure(figsize = (5, 5))
    plt.imshow(img, cmap = 'gray')
    plt.show()

In [16]:
# # MNIST 재현하기 with batch -> data load를 한번에 많이 해서 for문을 돌림 : for문 하나에 한개씩 돌리는 것보다빠름 : 500ms
# x, t = get_data()
# network = init_network()

# batch_size = 1000
# accuracy_cnt = 0

# for i in range(0, len(x), batch_size):
#     x_batch = x[i : i + batch_size]
#     y_batch = predict(network, x_batch)
#     p = np.argmax(y_batch, axis = 1)
    
#     accuracy_cnt += np.sum(p == t[i : i + batch_size]) # bool array 생성 후 1만 sum
        
# print("Accuracy:" + str(float(accuracy_cnt)/len(x)))

In [17]:
def mean_squared_error(y, t):
    return 0.5*np.sum((y-2)**2)

In [18]:
def cross_entropy_error(y, t): # one-hot encoding 아닌경우
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스 반환
    if t.size == y.size: # size는 배열 원소 갯수. len은 배열 맨 마지막 리스트 길이
        t = t.argmax(axis = 1)
        
    batch_size = y.shape[0]
    delta = 1e-7
    
    return -np.sum(np.log(y[np.arange(batch_size), t] + delta))/batch_size # y[0, 2] 등 정답에 해당하는 값만 log로 넘겨서 계산하게 함(list trick)

In [19]:
def numerical_diff(f, x):
    h = 1e-4
    return (f(x+h) - f(x-h)) / (2*h)

In [20]:
def function_1(x):
    return 0.01*x**2 + 0.1*x

In [21]:
def function_1_diff(x):
    return numerical_diff(function_1, 5)*x

In [22]:
def function_2(x):
    return np.sum(x**2)

In [23]:
def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)
    
#     for idx in range(x.size):
#         fxh1 = f(x[idx] + h)
#         fxh2 = f(x[idx] - h)
        
#         grad[idx] = (fxh1 - fxh2) / (2*h)

    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite']) # 다차원 배열 처리용으로 업그레이드
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        it.iternext()   
    
    return grad

In [24]:
def gradient_descent(f, init_x, lr = 0.01, step_num = 100):
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr*grad
        
    return x

In [25]:
class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2, 3) # 2 by 3 짜리 정규분포(~N(0,1)) 랜덤생성
        
    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        return loss